In [1]:
import pandas as pd
import numpy as np
from annoy import AnnoyIndex
from transformers import AutoTokenizer, TFBertModel
import matplotlib.pyplot as plt
import tensorflow as tf
import sys
from tqdm import tqdm

sys.path.append('../src')

In [28]:
products = pd.read_csv('../data/pareto_training.csv')
products = products.dropna(subset=['name']).reset_index(drop=True)

In [ ]:
scaler = StandardScaler()
scaler.fit(products.price.values)
products['price'] = scaler.transform(products.price.values)

In [2]:
lm = 'indobenchmark/indobert-base-p1'
model = TFBertModel.from_pretrained('../models/pareto_full_bert')
tokenizer = AutoTokenizer.from_pretrained(lm)

Some layers from the model checkpoint at ../models/pareto_full_bert were not used when initializing TFBertModel: ['classifier', 'dropout_37']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at ../models/pareto_full_bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [38]:
from utils.utils import make_dataset
encodings = tokenizer(products.name.tolist(), truncation=True, padding='max_length', max_length=150, return_attention_mask=True, add_special_tokens = True)
dataset = tf.data.Dataset.from_tensor_slices((
        dict(encodings),
    ))
features = model.predict(dataset.batch(32), batch_size=32, verbose=1).last_hidden_state

174/174 [==============================] - 56s 322ms/step


MemoryError: Unable to allocate 2.38 GiB for an array with shape (5541, 150, 768) and data type float32

In [ ]:
features = features[:, 0, :]

In [ ]:
index = AnnoyIndex(features.shape[1], 'angular')

for i in tqdm(range(features.shape[0])):
    vec = features[i]
    prod = products.loc[i]
#     vec = np.append(vec, prod.price)
    index.add_item(i, vec)
index.build(10)


In [ ]:
def product_nn(id):
    target_prod = products[products.id == id].iloc[0]
    

In [ ]:
from sklearn.manifold import TSNE
tnse_embedding = TSNE(n_components=2).fit_transform(embeddings)
plt.plot